In [30]:
import torch.nn.functional as F
import torch

In [31]:
y = torch.tensor([1.0])          #2
x1 = torch.tensor([1.1])    #3
w1 = torch.tensor([2.2])    #4
b = torch.tensor([0.0])            #5
z = x1 * w1 + b                 #6
a = torch.sigmoid(z)               #7
loss = F.binary_cross_entropy(a, y)

In [32]:
loss

tensor(0.0852)

In [33]:
w1.requires_grad

False

In [34]:
import torch.nn.functional as F
from torch.autograd import grad

y = torch.tensor([1.0])
x1 = torch.tensor([1.1])
w1 = torch.tensor([2.2], requires_grad=True)
b = torch.tensor([0.0], requires_grad=True)

z = x1 * w1 + b 
a = torch.sigmoid(z)

loss = F.binary_cross_entropy(a, y)

grad_L_w1 = grad(loss, w1, retain_graph=True)   #1
grad_L_b = grad(loss, b, retain_graph=True)

In [35]:
grad_L_w1

(tensor([-0.0898]),)

In [36]:
grad_L_b

(tensor([-0.0817]),)

In [37]:
print(grad_L_w1)
print(grad_L_b)

(tensor([-0.0898]),)
(tensor([-0.0817]),)


In [38]:
loss.backward()
print(w1.grad)
print(b.grad)

tensor([-0.0898])
tensor([-0.0817])


In [39]:
m = torch.nn.Linear(1, 2)

In [40]:
m.weight.shape, m.bias.shape

(torch.Size([2, 1]), torch.Size([2]))

In [41]:
m.weight

Parameter containing:
tensor([[ 0.7512],
        [-0.2546]], requires_grad=True)

In [42]:
m.bias

Parameter containing:
tensor([0.4412, 0.5694], requires_grad=True)

In [43]:
x = torch.randn(1, 1)
x.shape

torch.Size([1, 1])

In [44]:
x

tensor([[-0.6764]])

In [45]:
m(x)

tensor([[-0.0669,  0.7416]], grad_fn=<AddmmBackward0>)

In [46]:
0.8755*(-1.3004)+0.9037, 0.6011*(-1.3004)+0.6797

(-0.23480020000000001, -0.10197044)

In [47]:
class NeuralNetwork(torch.nn.Module):
    def __init__(self, num_inputs, num_outputs, size_hidden_1 = 30, size_hidden_2 = 20):    #1
        super().__init__()

        self.layers = torch.nn.Sequential(

            # 1st hidden layer
            torch.nn.Linear(num_inputs, size_hidden_1),    #2
            torch.nn.ReLU(),               #3

            # 2nd hidden layer
            torch.nn.Linear(size_hidden_1, size_hidden_2),    #4
            torch.nn.ReLU(),

            # output layer
            torch.nn.Linear(size_hidden_2, num_outputs),
        )

    def forward(self, x):
        logits = self.layers(x)
        return logits           #5

In [48]:
model = NeuralNetwork(50, 3)

In [49]:
model

NeuralNetwork(
  (layers): Sequential(
    (0): Linear(in_features=50, out_features=30, bias=True)
    (1): ReLU()
    (2): Linear(in_features=30, out_features=20, bias=True)
    (3): ReLU()
    (4): Linear(in_features=20, out_features=3, bias=True)
  )
)

In [50]:
model.parameters()

<generator object Module.parameters at 0x000001BA6CAF6CE0>

In [51]:
num_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
print("Total number of trainable model parameters:", num_params)

Total number of trainable model parameters: 2213


In [52]:
print(model.layers[0].weight.shape)

torch.Size([30, 50])


In [53]:
model.layers[0].weight[1].shape

torch.Size([50])

In [54]:
torch.manual_seed(123)
X = torch.rand((1, 50))
out = model(X)
print(out)

tensor([[-0.0281, -0.1572,  0.1557]], grad_fn=<AddmmBackward0>)


In [55]:
m = torch.nn.Softmax(dim=1)
x = abs(torch.randn(2, 3))
y = m(x)

In [56]:
y

tensor([[0.5543, 0.2162, 0.2295],
        [0.1798, 0.1740, 0.6462]])

In [57]:
x

tensor([[2.0975, 1.1558, 1.2157],
        [0.1295, 0.0967, 1.4086]])

In [58]:
X_train = torch.tensor([
    [-1.2, 3.1],
    [-0.9, 2.9],
    [-0.5, 2.6],
    [2.3, -1.1],
    [2.7, -1.5]
])
y_train = torch.tensor([0, 0, 0, 1, 1])

X_test = torch.tensor([
    [-0.8, 2.8],
    [2.6, -1.6],
])
y_test = torch.tensor([0, 1])

In [59]:
from torch.utils.data import Dataset

class ToyDataset(Dataset):
    def __init__(self, X, y):
        self.features = X
        self.labels = y

    def __getitem__(self, index):        #1
        one_x = self.features[index]     #1
        one_y = self.labels[index]       #1
        return one_x, one_y              #1

    def __len__(self):
        return self.labels.shape[0]      #2

train_ds = ToyDataset(X_train, y_train)
test_ds = ToyDataset(X_test, y_test)

In [60]:
train_ds[-1]

(tensor([ 2.7000, -1.5000]), tensor(1))

In [61]:
from torch.utils.data import DataLoader

train_loader = DataLoader(
    dataset=train_ds,     #1
    batch_size=2,
    shuffle=True,          #2
    num_workers=0,     #3
    drop_last=True
)

test_loader = DataLoader(
    dataset=test_ds,
    batch_size=2,
    shuffle=False,     #4
    num_workers=0
)

In [62]:
for idx, (x, y) in enumerate(train_loader):
    print(f"Batch {idx+1}:", x, y)

Batch 1: tensor([[ 2.3000, -1.1000],
        [-0.9000,  2.9000]]) tensor([1, 0])
Batch 2: tensor([[ 2.7000, -1.5000],
        [-1.2000,  3.1000]]) tensor([1, 0])


In [63]:
import torch.nn.functional as F

model = NeuralNetwork(num_inputs=2, num_outputs=2)    #1
optimizer = torch.optim.SGD(model.parameters(), lr=0.5)            #2

num_epochs = 3
for epoch in range(num_epochs): 

    model.train()
    for batch_idx, (features, labels) in enumerate(train_loader):
        logits = model(features)

        loss = F.cross_entropy(logits, labels)

        optimizer.zero_grad()            #3
        loss.backward()         #4
        optimizer.step()        #5

        ### LOGGING
        print(f"Epoch: {epoch+1:03d}/{num_epochs:03d}"
              f" | Batch {batch_idx:03d}/{len(train_loader):03d}"
              f" | Train Loss: {loss:.2f}")

    model.eval()
    # Insert optional model evaluation code

Epoch: 001/003 | Batch 000/002 | Train Loss: 0.78
Epoch: 001/003 | Batch 001/002 | Train Loss: 1.29
Epoch: 002/003 | Batch 000/002 | Train Loss: 0.30
Epoch: 002/003 | Batch 001/002 | Train Loss: 0.00
Epoch: 003/003 | Batch 000/002 | Train Loss: 0.00
Epoch: 003/003 | Batch 001/002 | Train Loss: 0.00


In [64]:
def compute_accuracy(model, dataloader):

    model = model.eval()
    correct = 0.0
    total_examples = 0

    for idx, (features, labels) in enumerate(dataloader):

        with torch.no_grad():
            logits = model(features)

        predictions = torch.argmax(logits, dim=1)
        compare = labels == predictions       #1
        correct += torch.sum(compare)      #2
        total_examples += len(compare)

    return (correct / total_examples).item()    #3

In [65]:
compute_accuracy(model, train_loader)

1.0

In [66]:
compute_accuracy(model, test_loader)

1.0

In [68]:
print(torch.cuda.is_available())

False


In [70]:
(1+2+10-3)/4, (1+2)/4 +(10-3)/4

(2.5, 2.5)